### Fundamentals of Natural Language Processing
# Negation and Uncertainty Detection using a Machine-Learning Based Approach

*Authors:*

> *Anna Blanco, Agustina Lazzati, Stanislav Bultaskii, Queralt Salvadó*

*Aims:*
> Rewrite for DL

In [5]:
# Import necessary libraries and functions
import json
import spacy
from collections import defaultdict
import re
import pandas as pd
from preprocessing import df_svm_neg_test, df_svm_neg_train, df_svm_neg_test, df_svm_unc_train, df_svm_unc_test, df_crf_neg_train, df_crf_neg_test, df_crf_unc_train, df_crf_unc_test
from sklearn_crfsuite import metrics

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
# We can reuse the previous dataset used for ML approach
df_neg_train = df_cf_neg_train.copy()
df_neg_test = df_crf_neg_test.copy()
df_unc_train = df_crf_unc_train.copy()
df_unc_test = df_crf_unc_test.copy()

In [ ]:
neg_cues = set()
unc_cues = set()

for _, row in df_svm_neg_train.iterrows():
    if row['neg_cue_label'] == 1:
        neg_cues.add(row['word'].lower())

for _, row in df_svm_unc_train.iterrows():
    if row['unc_cue_label'] == 1:
        unc_cues.add(row['word'].lower())

In [ ]:
def to_bio_labels(labels, label_type="SCOPE"):
    # Convert lists of binary labels (0/1) into BIO tagging format with custom suffixes
    bio_labels = []

    # Map original suffixes to new ones
    if "neg" in label_type.lower():
        prefix = "NSCO"   # Instead of NEG_SCOPE
    elif "unc" in label_type.lower():
        prefix = "UNSCO"  # Instead of UNC_SCOPE
    else:
        prefix = label_type.upper()  # fallback

    for sent in labels:
        bio = []
        prev = 0
        for i, tag in enumerate(sent):
            if tag == 1:
                if i == 0 or prev == 0:
                    bio.append(f'B-{prefix}')
                else:
                    bio.append(f'I-{prefix}')
            else:
                bio.append('O')
            prev = tag
        bio_labels.append(bio)
    return bio_labels

def prepare_data(df_train, df_test, label_col):
    scope_type = "NEG" if "neg" in label_col.lower() else "UNC"

    X_train = df_to_crf_format(df_train, scope_type)
    y_train_raw = df_to_labels(df_train, label_col)
    y_train = to_bio_labels(y_train_raw, label_type=scope_type)

    X_test = df_to_crf_format(df_test, scope_type)
    y_test_raw = df_to_labels(df_test, label_col)
    y_test = to_bio_labels(y_test_raw, label_type=scope_type)

    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_text, y_test = prepare_data(df_neg_train, df_neg_test, "neg_scope_label")
print(x_train)